In this notebook, I will look at the various files generated by the fineSTRUCTURE run on chr8

In [1]:
%run ../scripts/notebook_settings.py
from sklearn.decomposition import PCA
from sklearn import preprocessing
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [2]:
chunkcounts_path = "/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/fs/chr8_run/chr8_run_linked.chunkcounts.out"
mcmcmatrix_path = "/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/fs/chr8_run/mcmcmatrix_chr8.txt"
metadata_path = "../data/Papio_metadata_with_clustering.txt"

In [3]:
chunkcounts = pd.read_csv(chunkcounts_path, sep=" ", header = 1, index_col=0)
mcmcmatrix = pd.read_csv(mcmcmatrix_path, sep=",", index_col=0)
meta_data_samples = pd.read_csv(metadata_path, sep =" ")
meta_data_samples_Sci = meta_data_samples.copy()
for i, row in meta_data_samples_Sci.iterrows():
    if row.PGDP_ID[0] != "P":
        meta_data_samples_Sci.at[i, "PGDP_ID"] = "Sci_"+str(row.PGDP_ID)

Determining clusters by considering dendrogram, PCA, heatmap and sampling location.

In [4]:
meta_data_samples_Sci.index = meta_data_samples_Sci.PGDP_ID

In [5]:
kindae = meta_data_samples_Sci.reindex(mcmcmatrix.index)[:27].C_origin
cyno_western = meta_data_samples_Sci.reindex(mcmcmatrix.index)[27:37].C_origin
ursinus = meta_data_samples_Sci.reindex(mcmcmatrix.index)[37:41].C_origin
cyno_mikumi = meta_data_samples_Sci.reindex(mcmcmatrix.index)[41:78].Origin
cyno_central = meta_data_samples_Sci.reindex(mcmcmatrix.index)[78:93].Origin #1 Mikumi is also in here
anubis_gombe = meta_data_samples_Sci.reindex(mcmcmatrix.index)[93:110].Origin
anubis_kenya = meta_data_samples_Sci.reindex(mcmcmatrix.index)[110:112].Origin
anubis_serengeti = meta_data_samples_Sci.reindex(mcmcmatrix.index)[112:126].Origin
anubis_manyara_ngorongoro = meta_data_samples_Sci.reindex(mcmcmatrix.index)[126:151].Origin
anubis_arusha_tarangire = meta_data_samples_Sci.reindex(mcmcmatrix.index)[151:162].Origin
anubis_ethiopia = meta_data_samples_Sci.reindex(mcmcmatrix.index)[162:187].Origin
gelada_captive = meta_data_samples_Sci.reindex(mcmcmatrix.index)[187:189].Origin
papio_senegal = meta_data_samples_Sci.reindex(mcmcmatrix.index)[189:201].Origin
hamadryas_ethiopia = meta_data_samples_Sci.reindex(mcmcmatrix.index)[201:].Species

In [6]:
pop_df = pd.DataFrame()
pop_df["ID"] = meta_data_samples_Sci.reindex(mcmcmatrix.index).PGDP_ID
pop_df["population"] = "unknown"
pop_df["inclusion"] = "unknown"
pop_df.iloc[0:27,1], pop_df.iloc[0:27,2] = "kindae", 1
pop_df.iloc[27:37,1], pop_df.iloc[27:37,2] = "cyno_western", 1
pop_df.iloc[37:41,1], pop_df.iloc[37:41,2] = "ursinus", 1
pop_df.iloc[41:78,1], pop_df.iloc[41:78,2] = "cyno_mikumi", 1
pop_df.iloc[78:93,1], pop_df.iloc[78:93,2] = "cyno_RS", 1
pop_df.iloc[93:110,1], pop_df.iloc[93:110,2] = "anubis_gombe", 1
pop_df.iloc[110:112,1], pop_df.iloc[110:112,2] = "anubis_kenya", 0
pop_df.iloc[112:126,1], pop_df.iloc[112:126,2] = "anubis_serengeti", 1
pop_df.iloc[126:162,1], pop_df.iloc[126:162,2] = "anubis_MGAT", 1
pop_df.iloc[162:187,1], pop_df.iloc[162:187,2] = "anubis_ethiopia", 1
pop_df.iloc[187:189,1], pop_df.iloc[187:189,2] = "gelada", 0
pop_df.iloc[189:201,1], pop_df.iloc[189:201,2] = "papio", 1
pop_df.iloc[201:,1], pop_df.iloc[201:,2] = "hamadryas", 1

In [7]:
pop_df = pop_df.reindex(meta_data_samples_Sci.index)
pop_df.to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_fs_cluster.ids",
              sep=" ", header=False, index=False)

Subset for only the females in the chrX vcf. Alan has followed my gendering, so that is nice.

In [8]:
vcf_x = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/chrX/chrX.phased.rehead.vcf.gz"
callset = allel.read_vcf(vcf_x)
pop_df_females = pop_df.reindex(callset["samples"])

In [12]:
pop_df_females.to_csv("/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/chrX_females.ids",
              sep=" ", header=False, index=False)

In [9]:
pop_df_females

,ID,population,inclusion
PGDP_ID,,,
Sci_16066,Sci_16066,cyno_mikumi,1
Sci_16098,Sci_16098,cyno_mikumi,1
Sci_30977,Sci_30977,anubis_kenya,0
Sci_34449,Sci_34449,kindae,1
Sci_34474,Sci_34474,kindae,1
...,...,...,...
PD_0788,PD_0788,kindae,1
PD_0789,PD_0789,kindae,1
PD_0790,PD_0790,kindae,1


In [10]:
chunkcounts.loc[chunkcounts.max() > 4000]

,Sci_16066,Sci_16098,Sci_30877,Sci_30977,Sci_34449,Sci_34474,Sci_38168,Sci_97074,Sci_97124,PD_0067,...,PD_0785,PD_0786,PD_0787,PD_0788,PD_0789,PD_0790,PD_0791,PD_0792,PD_0793,PD_0794_BAB
Recipient,,,,,,,,,,,,,,,,,,,,,
Sci_38168,13.168770,15.843611,9.438110,10.618792,21.212816,21.165529,0.00000,10.915043,12.595547,4456.85035,...,14.772215,21.242087,19.059621,16.579871,17.214142,21.097534,15.314339,19.649880,16.468717,18.703289
PD_0067,13.999878,13.318741,9.699941,10.741143,13.442803,15.844821,5194.56355,9.195348,9.391233,0.00000,...,15.253415,18.517194,17.025187,12.827421,16.295477,16.044948,21.018025,17.076559,16.641086,14.036267


PCA

In [11]:
in_df = meta_data_samples_Sci.loc[meta_data_samples_Sci["PGDP_ID"].isin(chunkcounts.index)]
sorted_origin = in_df.sort_values(by=["C_origin"])
sorted_chunks = chunkcounts.reindex(sorted_origin["PGDP_ID"])
sorted_chunks = sorted_chunks.reindex(columns=sorted_origin["PGDP_ID"])
merged_chunk_meta = pd.merge(sorted_chunks, in_df, on="PGDP_ID")

ValueError: 'PGDP_ID' is both an index level and a column label, which is ambiguous.

In [ ]:
pca = PCA(n_components=10)
X_normalized = preprocessing.normalize(sorted_chunks, norm='l2')
projection = pca.fit_transform(X_normalized)

print(pca.explained_variance_)

In [ ]:
pcx, pcy = 1, 0
g = sns.scatterplot(x=projection[:,pcx], y=projection[:,pcy], hue=merged_chunk_meta["C_origin"])
g.legend(bbox_to_anchor=(1.02, 1.02))
g.set(xlabel="pc{}".format(pcx+1), ylabel="pc{}".format(pcy+1))

I can either generate clusters based on chunkcounts, or try to extract the tree generated by fineSTRUCTURE and analyze that. I will first try to generate it based on chunkcounts/PCA results

In [ ]:
chunkpercentage = chunkcounts.div(chunkcounts.sum(axis=1), axis=0)

For each individual, I will make a list of individuals over a certain threshold of relatedness.